### Импортируем необходимые нам библиотеки

In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
from datetime import datetime

# загрузим специальную библиотеку, позволяющую решать оптимизационные задачи линейного программирования
from scipy.optimize import linprog

#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

### Скачиваем с rp5.ru текущий прогноз внешней температуры на 24 часа вперед

In [2]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Edg/95.0.1020.53'}
urllib3.disable_warnings()

In [3]:
# сайт метеостанции на ВДНХ г. Москва
url = 'https://rp5.ru/%D0%9F%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0_%D0%B2_%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B1%D1%83%D1%80%D0%B3%D0%B5'

response = req.get(url, headers=header, timeout=50, verify=False)
page = BeautifulSoup(response.text,'html.parser')

In [4]:
temperatures = page.find('table', {'id': ['forecastTable_1_3']}).find_all('td', {'class': ['n underlineRow toplineRow red',
                                              'n2 underlineRow toplineRow red',
                                              'd underlineRow toplineRow red',
                                              'd2 underlineRow toplineRow red',
                                              'n underlineRow toplineRow blue',
                                              'n2 underlineRow toplineRow blue',
                                              'd underlineRow toplineRow blue',
                                              'd2 underlineRow toplineRow blue']})

In [5]:
t = []

for temp in temperatures:
    t.append(temp.find('div', class_='t_0').text)

In [7]:
outside_temperature = []

for i in range(24):
    outside_temperature.append(int(t[i]))
    print(f'tвнеш{i} = {outside_temperature[i]} град.')

tвнеш0 = 13 град.
tвнеш1 = 13 град.
tвнеш2 = 13 град.
tвнеш3 = 12 град.
tвнеш4 = 11 град.
tвнеш5 = 10 град.
tвнеш6 = 8 град.
tвнеш7 = 7 град.
tвнеш8 = 7 град.
tвнеш9 = 5 град.
tвнеш10 = 4 град.
tвнеш11 = 4 град.
tвнеш12 = 2 град.
tвнеш13 = 1 град.
tвнеш14 = -1 град.
tвнеш15 = -2 град.
tвнеш16 = -3 град.
tвнеш17 = -4 град.
tвнеш18 = -5 град.
tвнеш19 = -5 град.
tвнеш20 = -4 град.
tвнеш21 = -2 град.
tвнеш22 = 0 град.
tвнеш23 = 1 град.


### Температурный график

In [8]:
temps_out = np.zeros(34)
temps_out[0] = 8

for i in range(33):
    temps_out[i+1] = temps_out[i] - 1

In [9]:
temps_out

array([  8.,   7.,   6.,   5.,   4.,   3.,   2.,   1.,   0.,  -1.,  -2.,
        -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9., -10., -11., -12., -13.,
       -14., -15., -16., -17., -18., -19., -20., -21., -22., -23., -24.,
       -25.])

In [10]:
temps_heat = np.array([75, 75, 75, 75, 75, 75, 75, 77, 81, 84, 86, 89, 92, 95, 97, 100, 103, 106, 109, 111, 114, 117, 120, 122, 125, 128, 128, 128, 128, 128, 128, 128, 128, 128])
Qmax_heat = np.array([2.98, 2.98, 2.98, 2.98, 2.98, 2.98, 2.98, 3.1, 3.35, 3.53, 3.65, 3.84, 4.03, 4.21, 4.33, 4.52, 4.72, 4.91, 5.1, 5.23, 5.43, 5.62, 5.82, 5.92, 6.15, 6.34, 6.34, 6.34, 6.34, 6.34, 6.34, 6.34, 6.34, 6.34])

### Описание постановки оптимизационной задачи

Для целей поиска оптимального графика подачи тепла используем зависимость температуры внутри помещения от времени остывании и нагревании здания:

$$
\Large{t}_в = {t}_н + \frac{Q}{Vx} + \frac{({t^{'}}_в - {t}_н - \frac{Q}{Vx})}{e^{(\frac{z}{\gamma})}}
$$

- $\Large{t}_в$ - температура, которая установится через z часов после нарушения теплового режима;
- $\Large{t^{'}}_в$ - температура, которая была в помещении на момент нарушения теплового режима;
- $\Large{t}_н$ - температура наружного воздуха;
- Q - количество тепла, подаваемого в помещение, Дж/час;
- V - объем здания по наружному обмеру, $м^{3}$;
- x -  отопительная характеристика здания, $\Large\frac{Дж}{м^{3}ч^{o}C}$;
- $\Large\gamma$ - коэффициент аккумуляции, характеризующий аккумулирующую способность наружных ограждений, ч.

Задача состоит в том, чтобы, зная температурный профиль наружного воздуха на несколько периодов времени вперед, рассчитать график подачи тепла в эти периоды с минимумом суммы поданного тепла за эти периоды.

Математически это означает, что целевая функция оптимизационной задачи является следующей суммой:

$$
\Large{Q} = {Q}_0 + {Q}_1 + ..... + {Q}_n  --> min
$$
где:
- n - количество прогнозных периодов.

или в матричном виде:

$$
\Large с^{T}\hat{Q}  --> min
$$
где:
- $\Large c$ - единичный вектор размерности n;
- $\Large\hat{Q}$ - вектор значений подачи тепла в разные периоды времени размерности n.

Исходя из формулы, учитывающей инерцию здания в остывании и нагревании, к целевой функции необходимо добавить следующие ограничения на $\Large {Q}_i$:

$$
\Large t_{н i-1} + \frac{Q_{i-1}}{Vx} + \frac{(t_{i-1} - t_{н i-1} - \frac{Q_{i-1}}{Vx})}{e^{(\frac{z}{\gamma})}} = t_{i} >= 16
$$

или упростив:

$$
\Large t_{i-1}\alpha + t_{н i-1}(1-\alpha) + Q_{i-1}\beta >= 16
$$
$$
\Large t_{i-1}\alpha + Q_{i-1}\beta >= 16 - t_{н i-1}(1-\alpha)
$$

где:
- $\Large \alpha = \frac{1}{e^{(\frac{z}{\gamma})}}$
- $\Large \beta = \frac{1 - \alpha}{qV}$

В ограничениях на Qi-1 все еще присутствует значение температуры в предыдущий период ti-1.</br>
Однако это значение можно также выразить через значение ti-2 и Qi-2.</br>
Таким образом, можно рекурсивно дойти до известной нам температуры внутри помещения в нулевой период t0.</br>
После рекурсии до температуры t0 ограничения на Qi будут выглядеть следующим образом:

$$
\Large Q_{0}*\beta*\alpha^{i-1} + Q_{1}*\beta*\alpha^{i-2} + ... + Q_{i}*\beta*\alpha^{0} >= 16 - t_{0}*\alpha{i} - t_{н0}*(1-\alpha)*alpha^{i-1} - ... - t_{нi}*(1-\alpha)*alpha^{0}
$$

Или в матричном виде ограничения записываются как:

$$
\Large A*\hat{Q} >= \hat{b}
$$
где:
- $\Large\hat{Q}$ - вектор значений подачи тепла в прогнозные периоды времени;
- $\Large{A}$ - матрица ограничений вида $\begin{bmatrix} \beta & 0 & 0 & ... & 0 \\ \beta*\alpha & \beta & 0 & ... & 0 \\\ ... & .. & .. & ... & .. \\\\ \beta*\alpha^{n-1} & .. & .. & ... & \beta \end{bmatrix}$
- $\Large\hat{b}$ - вектор вида $\begin{bmatrix} t_{min} - t_{0}*\alpha - t_{н0}*(1-\alpha) \\ t_{min} - t_{0}*\alpha^{2} - t_{н0}*(1-\alpha)*\alpha - t_{н1}*(1-\alpha) \\\ ..................................... \\\\ t_{min} - t_{0}*\alpha^{n} - t_{н0}*(1-\alpha)*\alpha^{n-1} - ... t_{нn}*(1-\alpha)*alpha^{0} \end{bmatrix}$

Мы определили целевую функцию и ограничения на ее переменные в виде матрицы и вектора. Теперь мы можем решить задачу линейного программирования (оптимизация такого рода выбрана по причине линейности как целевой функции, так и ограничений)

### Зададим характеристики здания, начальную температуру внутри помещения и ограничение по ней

In [11]:
# характеристика здания
qV = 0.1133

# количество часов, необходимое зданию для остывания в e раз (2,71 раз)
gamma = 40

alpha = np.round(np.exp(-1/gamma),4)
betta = np.round((1 - alpha)/qV,4)

# начальная температура внутри помещения
t0 = 17.0
# нижняя граница внутренней температуры в здании
tmin1 = 16.5
tmin2 = 14.5

# ограничения на минимальный и максимальный уровень подачи тепла
Qmin = 0.5

In [12]:
Qmax = np.zeros(24, dtype=float)
for i in range(24):
    if outside_temperature[i] <= 8:
        position = np.where(temps_out == outside_temperature[i])[0].tolist()[0]
        Qmax[i] = Qmax_heat[position]
    else:
        Qmax[i] = Qmax_heat[0] 

Qmax = Qmax.tolist()

In [13]:
Qmax

[2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 2.98,
 3.1,
 3.53,
 3.65,
 3.84,
 4.03,
 4.21,
 4.21,
 4.03,
 3.65,
 3.35,
 3.1]

In [14]:
alpha, betta

(0.9753, 0.218)

### Создадим ряд прогнозных часов

In [15]:
base = datetime.today()
hours = [(base + timedelta(hours=x)).hour for x in range(24)]

### Нахождение оптимального профиля подачи тепла

Для решения оптимизационной задачи, нам, используя характеристики здания, нач температуру и ограничение по температуре, необходимо найти вектор C, матрицу ограничений А и вектор ограничений b.

In [16]:
# вектор с - это просто единичный вектор
c = np.ones(24, dtype=int)
c

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [17]:
A =np.diag(np.full(24,-betta))
for i in range(1,24):
    A += np.diag(np.full(24-i,-betta*(alpha**i)), -i)

In [18]:
# матрица A получилась в необходимом нам виде
A.shape

(24, 24)

In [19]:
#создадим вектор ограничений b, сделаем его из двух частей, которые сложим вместе, так как это упростит создание циклов
b_1 = np.zeros(24)
b_1[0] = outside_temperature[0]*(1 - alpha)
for i in range(1,24):
    b_1[i] = b_1[i-1] + outside_temperature[i]*(1 - alpha)*alpha**i

In [20]:
#вторая часть вектора b
b_2 = np.zeros(24)
for i in range(24):
    if 19 <= hours[i] <= 24 or 0 <= hours[i] <= 6:
        b_2[i] = -tmin2 + t0*alpha**(i+1)
    else:
        b_2[i] = -tmin1 + t0*alpha**(i+1)

In [21]:
# складываем обе части вместе
b = np.round(b_1 + b_2, 2)
b

array([ 0.4 ,  0.3 ,  0.21,  0.1 , -0.04, -0.19, -0.38, -0.59, -0.79,
        0.97,  0.72,  0.48,  0.2 , -0.08, -0.4 , -0.72, -1.05, -1.39,
       -1.73, -2.07, -2.39, -4.66, -4.91, -5.13])

In [22]:
bounds = []

for i in range(24):
    bounds.append((Qmin, Qmax[i]))

len(bounds)

24

In [23]:
# проводим поиск оптимального решения
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds)

In [24]:
Qsum = np.round(result.fun,2)
print(f'Итого подача тепла за весь прогнозный период: {Qsum} Гкал')

Итого подача тепла за весь прогнозный период: 27.53 Гкал


In [25]:
#посмотрим на вектор Q
Q = np.round(np.array(result.x),2)
Q

array([0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 3.4 , 4.21, 4.03, 3.65,
       1.67, 1.57])

In [26]:
# расчитаем при таком профиле подачи тепла, как ведет себя температура внутри помещения
internal_temp = []
internal_temp.append(t0)

for i in range(1,24):
    ti = internal_temp[i-1]*alpha + Q[i-1]*betta + outside_temperature[i-1]*(1-alpha)
    internal_temp.append(np.round(ti,1))

In [27]:
# также рассчитаем температуру теплоносителя в подающем трубопроводе после смешения
heat_temp = []

for i in range(24):
    if 19 <= hours[i] <= 24 or 0 <= hours[i] <= 6:
        ti = tmin2 + ((115+80)/2 - tmin2)*(Q[i]/(qV*(tmin2+40)))**0.8 + ((115-80)/2)*(Q[i]/(qV*(tmin2+40)))
    else:
        ti = tmin1 + ((115+80)/2 - tmin1)*(Q[i]/(qV*(tmin1+40)))**0.8 + ((115-80)/2)*(Q[i]/(qV*(tmin1+40)))
    heat_temp.append(np.round(ti))

### Создадим отчетный файл с результатами расчетов

In [28]:
# создадим ряд прогнозных дат
dates = np.asarray([(base + timedelta(hours=x)).strftime('%Y-%m-%d %H') for x in range(24)]).reshape(24,1)

In [29]:
# преобразуем список температур внешнего воздуха в массив
out_tem = np.asarray(outside_temperature).reshape(24,1)

In [30]:
# преобразуем Q_Гкал, tподающ, tвнутр в массив
Q_Gkal = np.asarray(Q).reshape(24,1)
tpod = np.asarray(heat_temp).reshape(24,1)
tvn = np.asarray(internal_temp).reshape(24,1)

data_array = np.hstack((dates, out_tem, Q_Gkal, tpod, tvn))

In [31]:
# создадим перечень колонок в выходном файле csv
cols = ['дата', 'tвнеш', 'Q_Гкал', 'tподающ', 'tвнутр']

data = pd.DataFrame(data_array, columns=cols)
data

,дата,tвнеш,Q_Гкал,tподающ,tвнутр
0,2023-04-06 10,13,0.5,28.0,17.0
1,2023-04-06 11,13,0.5,28.0,17.0
2,2023-04-06 12,13,0.5,28.0,17.0
3,2023-04-06 13,12,0.5,28.0,17.0
4,2023-04-06 14,11,0.5,28.0,17.0
5,2023-04-06 15,10,0.5,28.0,17.0
6,2023-04-06 16,8,0.5,28.0,16.9
7,2023-04-06 17,7,0.5,28.0,16.8
8,2023-04-06 18,7,0.5,28.0,16.7
9,2023-04-06 19,5,0.5,27.0,16.6


In [40]:
data = pd.read_csv('C:/Users/Алексей Третьяков/Desktop/ML_rep/heat_project/output/data_2022_12_31_18_42.csv', encoding='cp1251', sep=',')
data

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Алексей Третьяков/Desktop/ML_rep/heat_project/output/data_2022_12_31_18_42.csv'

In [41]:
inputs = pd.read_csv('C:/Users/Алексей Третьяков/Desktop/ML_rep/heat_project/input/input_data.csv', encoding='cp1251')
inputs['tвнутр'].values[0]

17